In [ ]:
"""
Notebook for testing NotebookManager
"""

# Setup
import sys
import time
import os

import coverage

# Set path so this works for running live
cwd = os.getcwd()
if os.path.basename(cwd) == 'tests':
    project_dir = os.path.dirname(cwd)
    sys.path.insert(1, project_dir)
else:
    project_dir = cwd

# Start coverage, should be before imports
cov = coverage.Coverage(
    data_file=os.path.join(project_dir, '.coverage.notebook'),
    config_file=os.path.join(project_dir, 'setup.cfg')
)
cov.start()

# pylint: disable=wrong-import-position
from enlighten import get_manager, NotebookManager  # noqa: E402

In [ ]:
# test_get_manager
# Test we get the right manager when running in a notebook
manager = get_manager()
assert isinstance(manager, NotebookManager)
assert repr(manager) == 'NotebookManager()'

In [ ]:
# test_standard
# Test standard manager
manager = NotebookManager()
ctr = manager.counter(total=100)
ctr.update(force=True)
manager.stop()

In [ ]:
# test_disabled
# Test manager disabled
manager = NotebookManager(enabled=False)
ctr = manager.counter(total=100)
ctr.update(force=True)
manager.write('We should never see this')
manager.stop()

In [ ]:
# test_bare_no_flush
# Test write bare message, no flush

manager = NotebookManager()
manager.write('test message', flush=False)
# pylint: disable=protected-access
assert manager._output[0] == '  <div class="enlighten-bar">\n    <pre>test message</pre>\n  </div>'

In [ ]:
# test_advanced
# More advanced use case
manager = NotebookManager()

ticks = manager.counter(total=10, desc='Ticks', unit='ticks', color='red')
tocks = manager.counter(total=5, desc='Tocks', unit='tocks', color='blue', position=3)

for num in range(10):
    time.sleep(0.01)  # Simulate work
    ticks.update()
    if not num % 2:
        tocks.update()
manager.stop()

In [ ]:
# test_styles
# Styles converted to HTML
manager = NotebookManager()
term = manager.term

status = manager.status_bar(' '.join((
    'normal',
    term.blue_on_aquamarine('blue_on_aquamarine'),
    term.aquamarine_on_blue('aquamarine_on_blue'),
    term.color(90)('color_90'),
    term.on_color(90)('on_color_90'),
    term.italic_bright_red('italics_red'),
    term.on_bright_blue('on_bright_blue'),
    term.blink('blink'),
    term.bold('bold'),
    term.bold(''),  # Empty span will be ignored
    term.underline('underline'),
    term.reverse('unsupported_reverse'),
    term.move(5, 6) + 'unsupported_move',
    term.normal + 'ignore_unmatched_normal',
    term.link('https://pypi.org/project/enlighten/', 'enlighten'),
)))

In [ ]:
# Cleanup
cov.stop()
cov.save()